# クラウドでのデータサイエンス: "Azure ML SDK" の方法

## はじめに

このノートブックでは、Azure ML SDKを使用してモデルをトレーニング、デプロイ、そして消費する方法を学びます。

事前準備:
1. Azure MLワークスペースを作成しました。
2. [心不全データセット](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data)をAzure MLにロードしました。
3. このノートブックをAzure ML Studioにアップロードしました。

次のステップは以下の通りです:

1. 既存のワークスペースで実験を作成する。
2. コンピュートクラスターを作成する。
3. データセットをロードする。
4. AutoMLConfigを使用してAutoMLを設定する。
5. AutoML実験を実行する。
6. 結果を確認し、最適なモデルを取得する。
7. 最適なモデルを登録する。
8. 最適なモデルをデプロイする。
9. エンドポイントを消費する。

## Azure Machine Learning SDK固有のインポート


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## ワークスペースの初期化  
保存された設定からワークスペースオブジェクトを初期化します。設定ファイルが .\config.json に存在していることを確認してください。  


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Azure ML 実験を作成する

先ほど初期化したワークスペース内に、'aml-experiment' という名前の実験を作成しましょう。


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## コンピュートクラスターを作成する
AutoML の実行には、[コンピュートターゲット](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target)を作成する必要があります。


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## データ
データセットをAzure MLにアップロードし、キーがデータセットと同じ名前であることを確認してください。


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## AutoML構成


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## AutoML 実行


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## 最適なモデルをデプロイする

以下のコードを実行して、最適なモデルをデプロイします。デプロイの状態は Azure ML ポータルで確認できます。このステップには数分かかる場合があります。


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## エンドポイントの利用
以下の入力サンプルに入力を追加することができます。


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**免責事項**:  
この文書は、AI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書を正式な情報源としてお考えください。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当社は一切の責任を負いません。
